In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import time
start_time = time.time() # Count the time

# Constants
a1 = 50
a2 = 100
b = 1.00
k1 = k2 = 0.1 # this is more realistic
time_range = 300
t_steps = 300

# Define the ODE system
def simplePP(t, Y, t1, t3):
    dYdt = [
        b*Y[2] - a1*Y[0]*Y[3] / (k1 + Y[0]) - b*Y[2]*Y[0],
        a1*Y[0]*Y[3] / (k1 + Y[0]) - a2*Y[1]*Y[4] / (k2 + Y[1]) - b*Y[2]*Y[1],
        t1*a2*Y[1]*Y[4] / (k2 + Y[1]) - b*Y[2]*Y[2],
        (1 - t1 - t3)*a2*Y[1]*Y[4] / (k2 + Y[1]) - b*Y[2]*Y[3],
        t3*a2*Y[1]*Y[4] / (k2 + Y[1]) - b*Y[2]*Y[4]
    ]
    return dYdt

# Initial conditions
Y0 = [0.1, 0.2, 0.3, 0.3, 0.1]
# The fixed points of this system are independent of initial conditions, but for some systems they may BE initial-condition-dependent

t_eval = np.linspace(0, time_range, t_steps)

# Parameter ranges
t_resol = 200
t1_vals = np.linspace(0, 1, t_resol+1)
t3_vals = np.linspace(0, 1, t_resol+1)

Z_lambda = np.zeros((len(t1_vals), len(t3_vals)))
Z_b = np.zeros((len(t1_vals), len(t3_vals)))
Z_kappa = np.zeros((len(t1_vals), len(t3_vals)))
Z_r = np.zeros((len(t1_vals), len(t3_vals)))

# Solve ODEs for each (t1, t3) pair where t1 + t3 <= 1
for i, t1 in enumerate(t1_vals):
    for j, t3 in enumerate(t3_vals):
        if t1 + t3 > 1:
            Z_lambda[i, j] = np.nan  # Mask invalid points
            Z_kappa[i, j] = np.nan
            Z_r[i, j] = np.nan
            continue

        # Solve the system
        sol = solve_ivp(simplePP, [0, time_range], Y0, t_eval=t_eval, method='RK45', args=(t1, t3), rtol=1e-6, atol=1e-9) # Default: RK45, set relative and absolute error
        Y = sol.y[:, -1] # Final Y vector value

        Y1 = Y[0]
        Y2 = Y[1]
        Y3 = Y[2]
        Y4 = Y[3]
        Y5 = Y[4]

        # Compute lambda
        Z_lambda[i, j] = b*Y3
        if Z_lambda[i,j] == np.nanmax(Z_lambda):
            growth_rate = Z_lambda[i,j]
            partition = [t1, t3]

        # Compute kappa
        Z_kappa[i, j] = t1

        # Compute r_{avg}
        denominator = ((k1 + Y1) / (a1 * Y4)) + ((k2 + Y2) / (a2 * Y5))
        Z_r[i, j] = 1 / denominator if denominator != 0 else np.nan


theta1 = partition[0]
theta2 = 1 - partition[0] - partition[1]
theta3 = partition[1]

num_levels = 30
T1, T3 = np.meshgrid(t1_vals, t3_vals)

# Growth rate (contourf()) -> to show lines only, use contour()
levels = np.linspace(np.nanmin(Z_lambda), np.nanmax(Z_lambda), num_levels) # np.nanmin() and np.nanmax() ignores NaN values
plt.figure(figsize=(8, 6))
contour = plt.contourf(T3, T1, Z_lambda.T, levels=levels, cmap='jet')
plt.colorbar(contour, label=r'')
plt.xlabel(r'$\theta_3$')
plt.ylabel(r'$\theta_1$')
plt.title(r'Long-term growth rate $\lambda$')
plt.show()

# Plot Y(t)
sol = solve_ivp(simplePP, [0, time_range], Y0, t_eval=t_eval, method='RK45', args=(theta1, theta3)) # Default: RK45, solve with the optimised theta1, theta3
Y_star = sol.y

plt.figure(figsize=(8,6)) # set size of the graph (call before plt.plot)
plt.plot(t_eval, Y_star[0], lw=2, label=r'$Y_1(T)$ = ' + str(Y_star[0][-10])[:7])
plt.plot(t_eval, Y_star[1], lw=2, label=r'$Y_2(T)$ = ' + str(Y_star[1][-10])[:7])
plt.plot(t_eval, Y_star[2], lw=2, label=r'$Y_3(T)$ = ' + str(Y_star[2][-10])[:7])
plt.plot(t_eval, Y_star[3], lw=2, label=r'$Y_4(T)$ = ' + str(Y_star[3][-10])[:7])
plt.plot(t_eval, Y_star[4], lw=2, label=r'$Y_5(T)$ = ' + str(Y_star[4][-10])[:7])
plt.xlabel("time (s)", size=13)
plt.ylabel(r"Biomass fraction $Y_i(t)$", size=13)
plt.title('Biomass Fraction Evolution')
plt.legend()
plt.show()

'''
# Pathway probability
levels = np.linspace(np.nanmin(Z_kappa), np.nanmax(Z_kappa), num_levels)
plt.figure(figsize=(8, 6))
contour = plt.contourf(T3, T1, Z_kappa.T, levels=levels, cmap='jet')
plt.colorbar(contour, label='')
plt.xlabel(r'$\theta_3$')
plt.ylabel(r'$\theta_1$')
plt.title(r'$q_{\pi}$ (pathway probability for $X_3$)')
plt.show()
'''

# Overall transfer rate
levels = np.linspace(np.nanmin(Z_r), np.nanmax(Z_r), num_levels)
plt.figure(figsize=(8, 6))
contour = plt.contourf(T3, T1, Z_r.T, levels=levels, cmap='jet')
plt.colorbar(contour, label='')
plt.xlabel(r'$\theta_3$')
plt.ylabel(r'$\theta_1$')
plt.title(r'$r_{avg}$ (overall transfer rate)')
plt.show()

# Growth rate is the largest value of $\lambda$ over all $(\theta_1, \theta_2, \theta_3)$
print('The growth rate is approximately ' + str(growth_rate)[:7] + r', at partition (' + str(theta1)[:7] + ', ' + str(theta2)[:7] + ', ' + str(theta3)[:7] + ')')
print('b = ' + str(b))
print('\n')


print(str("--- %s seconds ---" % (time.time() - start_time)).center(100)) # Total run time